In [4]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

First check to see what tables exist in our database

In [3]:
%%sql
SELECT *
FROM sqlite_master
WHERE type='table'

Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


Check the first five rows in the facts table in the database

In [6]:
%%sql
SELECT * 
FROM facts
LIMIT 5;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


## Summary Statistics

### Mimimum population

In [11]:
%%sql

SELECT *
FROM facts
WHERE population = (
    SELECT MIN(population)
    FROM facts
)

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


### Maximum Population

In [13]:
%%sql

SELECT *
FROM facts
WHERE population = (
    SELECT MAX(population)
    FROM facts
)

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


### Minimum Population Growth

In [17]:
%%sql

SELECT *
FROM facts
WHERE population_growth = (
    SELECT MIN(population_growth)
    FROM facts
)

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
190,vt,Holy See (Vatican City),0,0,0,842,0.0,None,None,None
200,ck,Cocos (Keeling) Islands,14,14,0,596,0.0,None,None,None
207,gl,Greenland,2166086,2166086,None,57733,0.0,14.48,8.49,5.98
238,pc,Pitcairn Islands,47,47,0,48,0.0,None,None,None


### Maximum Population Growth

In [18]:
%%sql

SELECT *
FROM facts
WHERE population_growth = (
    SELECT MAX(population_growth)
    FROM facts
)

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
162,od,South Sudan,644329,None,None,12042910,4.02,36.91,8.18,11.47


## Exploring Outliers

### Minimum Population

In [22]:
%%sql

SELECT name
FROM facts
WHERE population = (
    SELECT MIN(population)
    FROM facts
)

Done.


name
Antarctica


### Maximum Population

In [23]:
%%sql

SELECT name
FROM facts
WHERE population = (
    SELECT MAX(population)
    FROM facts
)

Done.


name
World


## Recompute Summary Statistics 

From above analyses we have found out that the population of the whole world was among the records. Thus, when determining the maximum value, we ended up with a "country" that has over 7 billion in population. Antarctica was also included, making the minimum population value to be 0. In this part, I redo some of that while considering such outliers

### Minimum Population

In [37]:
%%sql

SELECT *
FROM facts
WHERE population = (
    SELECT MIN(population)
    FROM facts
    WHERE (name != 'World') AND (name != 'Antarctica')
)

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
238,pc,Pitcairn Islands,47,47,0,48,0.0,None,None,None


### Maximum Population

In [36]:
%%sql

SELECT *
FROM facts
WHERE population = (
    SELECT MAX(population)
    FROM facts
    WHERE (name != 'World') AND (name != 'Antarctica')
)

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
37,ch,China,9596960,9326410,270550,1367485388,0.45,12.49,7.53,0.44


### Minimum Population Growth

In [38]:
%%sql

SELECT *
FROM facts
WHERE population_growth = (
    SELECT MIN(population_growth)
    FROM facts
    WHERE (name != 'World') AND (name != 'Antarctica')
)

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
190,vt,Holy See (Vatican City),0,0,0,842,0.0,None,None,None
200,ck,Cocos (Keeling) Islands,14,14,0,596,0.0,None,None,None
207,gl,Greenland,2166086,2166086,None,57733,0.0,14.48,8.49,5.98
238,pc,Pitcairn Islands,47,47,0,48,0.0,None,None,None


### Maximum Population Growth

In [39]:
%%sql

SELECT *
FROM facts
WHERE population_growth = (
    SELECT MAX(population_growth)
    FROM facts
    WHERE (name != 'World') AND (name != 'Antarctica')
)

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
162,od,South Sudan,644329,None,None,12042910,4.02,36.91,8.18,11.47


### Average Population

In [44]:
%%sql

SELECT AVG(population) as avg_population
FROM facts
WHERE (name != 'World') AND (name != 'Antarctica')

Done.


avg_population
32377011.0125


### Average Area

In [46]:
%%sql

SELECT AVG(area) as avg_area
FROM facts
WHERE (name != 'World') AND (name != 'Antarctica')

Done.


avg_area
555093.546184739


## Finding Densely Populated Countries

### Countries with Above Average Population

In [51]:
%%sql
SELECT name
FROM facts
WHERE population > (
    SELECT AVG(population)
    FROM facts
    WHERE (name != 'World') AND (name != 'Antarctica')
) AND (name != 'World') AND (name != 'Antarctica')

Done.


name
Afghanistan
Algeria
Argentina
Bangladesh
Brazil
Burma
Canada
China
Colombia
"Congo, Democratic Republic of the"


### Area is Below Average

In [52]:
%%sql
SELECT name
FROM facts
WHERE area < (
    SELECT AVG(area)
    FROM facts
    WHERE (name != 'World') AND (name != 'Antarctica')
) AND (name != 'World') AND (name != 'Antarctica')

Done.


name
Albania
Andorra
Antigua and Barbuda
Armenia
Austria
Azerbaijan
"Bahamas, The"
Bahrain
Bangladesh
Barbados
